# Read Data and Setup Neural Net

In [1]:
import csv

from sklearn.neural_network import MLPClassifier

import numpy as np

from globus_setup import *


print("reading data")
Xpred = []
X = []
y = []
with open(CSV_NAME, "r") as f:
    csv_reader = csv.DictReader(f)

    training_fields = set(csv_reader.fieldnames) - {"day", "tas"}
    for row in csv_reader:
        # Remove rows which are missing values for any variable
        empty_fields = [f for f in row if row[f] == ""]
        if empty_fields:
            continue
        features = list(v for k, v in row.items() if k in training_fields)
        if int(row["day"]) == 42002:
            Xpred.append(features)
        else:
            X.append(features)
            y.append(row["tas"])


reading data


# Train Neural Net

In [2]:

clf = MLPClassifier(verbose=True, hidden_layer_sizes=(20, 20, 20))
clf.fit(X, y)

Iteration 1, loss = 9.88690271
Iteration 2, loss = 9.87022850
Iteration 3, loss = 9.83529814
Iteration 4, loss = 9.49661091
Iteration 5, loss = 9.05806882
Iteration 6, loss = 8.74422955
Iteration 7, loss = 8.44576899
Iteration 8, loss = 8.13116543
Iteration 9, loss = 7.79554441
Iteration 10, loss = 7.47130262
Iteration 11, loss = 7.19540629
Iteration 12, loss = 6.97186583
Iteration 13, loss = 6.78893357
Iteration 14, loss = 6.63563666
Iteration 15, loss = 6.50307892
Iteration 16, loss = 6.38624737
Iteration 17, loss = 6.28290727
Iteration 18, loss = 6.18841634
Iteration 19, loss = 6.10169710
Iteration 20, loss = 6.02410050
Iteration 21, loss = 5.94811982
Iteration 22, loss = 5.88137294
Iteration 23, loss = 5.82047756
Iteration 24, loss = 5.76130896
Iteration 25, loss = 5.70438499
Iteration 26, loss = 5.65236261
Iteration 27, loss = 5.60220123
Iteration 28, loss = 5.55650185
Iteration 29, loss = 5.51119236
Iteration 30, loss = 5.46902696
Iteration 31, loss = 5.43128497
Iteration 32, los

C:\Users\mathe\Documents\Code\KU\Spring_22\EECS_738\Project\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 20, 20), verbose=True)

In [3]:
import pickle
with open("model.p", "wb") as f:
    pickle.dump(clf, f)

# Predict

In [4]:
print(clf.predict(Xpred))

['0.3931162381341585']


C:\Users\mathe\Documents\Code\KU\Spring_22\EECS_738\Project\.venv\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
